In [ ]:
import requests,re
import numpy as np
from bs4 import BeautifulSoup
import pdfkit

In [ ]:
base_url = 'http://paulgraham.com'

In [ ]:
test = requests.get(base_url+'/articles.html')
content = BeautifulSoup(test.content,'html.parser')

In [ ]:
tables = content.findAll('table')

In [ ]:
req_table_idx = 0;

In [ ]:
for idx,table_content in enumerate(tables):
    if str(table_content).find('Essays') != -1:
        req_table_idx  = idx+1

In [ ]:
req_table = tables[req_table_idx]

In [ ]:
essay_owner = '<a href="http://paulgraham.com/index.html"><img src="https://sep.yimg.com/ca/I/paulgraham_2271_3232" width="410" vspace="0" hspace="0" height="45" border="0"></a><br/><br/>'
essay_title = '<img src="https://sep.yimg.com/ca/I/paulgraham_2272_1072" alt="Essays" width="57" vspace="0" hspace="0" height="18" border="0"><br/><br/>'

In [ ]:
menu = essay_owner+essay_title+req_table.decode_contents()

In [ ]:
rows = req_table.findAll('tr')

In [ ]:
essay_url_list = []
essay_title_list = []
for row in rows:
    try:
        if len(row.select('a')) > 0:
            essay_info = row.select('a')[0]
            essay_title_list.append(essay_info.decode_contents())
            essay_url_list.append(essay_info['href'])
    except:
            print('error')

In [ ]:
def essay_links_fix(essay_content,idx):
    et = essay_content
    if(idx>=0):
        ct = re.sub("(=\")(\w+).html",r'\g<1>'+base_url+r'/\g<2>.html',essay_content)
        et = re.sub(r'href="#(\w+)"',r'href="#\g<1>{}z"'.format(str(idx)),ct)
        et = re.sub(r'name="(\w+)"',r'name="\g<1>{}z"'.format(str(idx)),et)
        et = re.sub(r'#dddddd',r'#a2a2a2',et)
        et = re.sub(r'((<br.>){3,})<b>(.*?)</b>',r'<br/><br/><b>\g<3></b>',et)
    return et

In [ ]:
start_template = '''<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
</head>
<body>'''
end_template = '''</body></html>'''

In [ ]:
essay_list = [start_template+menu+end_template]

In [ ]:
def replace_url(essay_url,ref_id):
#     print('ref:',ref_id)
    modified_url = essay_url[:]
    if(essay_url.find('?')!=-1) : 
        key = re.findall(r'\/(\w+.\w+)\?',essay_url)[0]
        if key:
#             print('key:',key)
            modified_url = re.sub(\
    r'<a href=("https://sep.yimg.com/ty/cdn/paulgraham/{}.*?")'.format(key),
    r'<a href="{}"'.format('#'+ref_id),essay_list[0])
    else:
        modified_url = re.sub(r'<a href=("{}")>'.format(essay_url),\
                           r'<a href="{}">'.format('#'+ref_id),essay_list[0])
    return modified_url

In [ ]:
for idx,essay_url in enumerate(essay_url_list):
    try:
        print('adding essay no:'+str(idx+1)+' '+essay_title_list[idx])
#         req_url = base_url+'/'+essay_url
        if(len(re.findall(r'(http)',essay_url))==0):req_url = base_url+'/'+essay_url
        else:req_url=essay_url
#         print('essay url:',essay_url)
        ref_id = str(idx)+'_essay'      
        essay_list[0] = replace_url(essay_url,ref_id)
        essay_raw = requests.get(req_url)
        essay_parsed = BeautifulSoup(essay_raw.content,'html.parser')
        table_list = essay_parsed.findAll('table')
        
        if(len(table_list)>0): essay_content = table_list[1]
        else:essay_content = essay_parsed
        
        essay_content = essay_content.decode_contents()
        essay_content = essay_links_fix(essay_content,idx)
        if(essay_content.find('img')==-1):
#             print('idx:',idx)
            essay_content = '<img src="https://sep.yimg.com/ca/I/paulgraham_2272_1423"/>' + essay_content
        essay_content = re.sub(r'(<img .*?/>)',
            r'<a href="{}" id={} >'.format(req_url,ref_id)+'\g<1>'+'</a>',
            essay_content)
        formatted_content = start_template+\
            '<p class="break-before"><div id="title">'\
            + essay_content  + '</div>'+end_template
        essay_list.append(formatted_content)
    except:
        print('error in adding:',essay_title_list[idx])

In [ ]:
options = {
   'footer-center': '-[page]-',
    'footer-font-size':'8'
   }

In [ ]:
# pdfkit.from_string(' '.join([et,et2]),'earnet.pdf',options=options,css='test_css.css')
pdfkit.from_string(' '.join(essay_list),'pg_essay.pdf',options=options,css='file_css.css')